In [2]:
import tensorflow as tf
import pickle
import time
import numpy as np
from sentence_transformers import SentenceTransformer, util
import pickle
from collections import namedtuple
from datetime import datetime
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import hnswlib
import subprocess
import torch
from getPassageFromPID import DocContentManager

In [20]:
sentence_transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

with open('.\\combined_corpus_embeddings\\combined_corpus_embeddings', 'rb') as file:
    corpus_embeddings = pickle.load(file)
    corpus_embeddings = torch.tensor(corpus_embeddings)
d = DocContentManager()

C:\Users\asg41\AppData\Local\Temp\ipykernel_16628\2254235322.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  corpus_embeddings = torch.tensor(corpus_embeddings)


In [25]:
query = input("Enter your query: ")

cpp_executable_path = '.\\bm25.exe' 
command = [cpp_executable_path, query]
process = subprocess.run(command, capture_output=True, text=True)

relavant_docs = []

if process.returncode == 0:
    output_lines = process.stdout.strip().split('\n')

    # Process each line of output
    for line in output_lines:
        pid = int(line)
        relavant_docs.append(pid)
        #print(pid)
        #print(d.fetchPassageContent(pid))
    #print(process.stderr)
else:
    print("Error:", process.stderr)

print(relavant_docs)

[6028962, 5404297, 574269, 7939951, 8604281, 7652173, 1419328, 7754005, 4145286, 2707791, 3534084, 2197889, 3340638, 1094854, 4380247, 1419326, 4114244, 2999462, 5739129, 5896207, 8589064, 5424813, 4380253, 1049755, 7754003]


In [26]:
query_embeddings = []
for i in relavant_docs:
    if i<7999999:
        query_embeddings.append(corpus_embeddings[i])
    
final_passages = []
hits = util.semantic_search(query_embeddings, corpus_embeddings)
for hit in hits[0]:
    final_passages.append(int(hit['corpus_id']))

for pid in final_passages:
    print(str(pid)+" : ")
    print(d.fetchPassageContent(pid))
    print()

IndexError: index 8604281 is out of bounds for dimension 0 with size 7999999

In [9]:
print(d.fetchPassageContent(7130335))

Augmented reality. Augmented reality (AR) is a live direct or indirect view of a physical, real-world environment whose elements are augmented by computer-generated or extracted real-world sensory input such as sound, video, graphics or GPS data.


In [19]:
print(d.fetchPassageContent(3923587))

Page 6 of 28. aicpa.org/FRC. 1. Accounts Receivable Subledger Standards. The AR subledger standard format is intended to accommodate basic analysis of the AR balance. This may. include analysis of the levels of activity during a specified period, how much of the AR balance is liquidated.
